In [1]:
import sys
sys.path.insert(0, '/Users/waihengsoh/Desktop/Desktop – Wai Heng/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI')

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
!pip install shap

In [1]:
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

2023-04-06 09:14:06.120774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import shap

In [ ]:
from Functions.Historical_News import historical_news
from datetime import date, timedelta
news = historical_news.get_all_news(date(2021, 8, 1), date(2021, 10, 31))


In [ ]:
train_df, test_df = news

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv('/Users/waihengsoh/Desktop/Desktop – Wai Heng/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI/csv_files/news/train.csv')
test_df.to_csv('/Users/waihengsoh/Desktop/Desktop – Wai Heng/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI/csv_files/news/test.csv')

In [4]:
# news = historical_news.get_all_news(date(2021, 8, 1), date(2021, 10, 31))
train = pd.read_csv("../csv_files/news/train.csv")
train = train.loc[:, ['content', 'price_change']]
X_train = train['content'].tolist()
y_train = train['price_change'].tolist()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

test = pd.read_csv("../csv_files/news/test.csv")
test = test.loc[:, ['content', 'price_change']]
X_test = test['content'].tolist()
y_test = test['price_change'].tolist()

print(len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test))


47041 47041 5227 5227 3263 3263


In [5]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None,
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [6]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train[:47000], train[47000:], 'content', 'price_change')

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


In [7]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, epsilon=3e-07, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=5, steps_per_epoch=100, validation_data=validation_data)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
100/100 [==============================] - 5346s 53s/step - loss: 0.6934 - accuracy: 0.5669 - val_loss: 0.7138 - val_accuracy: 0.5309
Epoch 2/5
100/100 [==============================] - 5400s 54s/step - loss: 0.6662 - accuracy: 0.6134 - val_loss: 0.6909 - val_accuracy: 0.5311
Epoch 3/5
100/100 [==============================] - 5366s 54s/step - loss: 0.6917 - accuracy: 0.5437 - val_loss: 0.6940 - val_accuracy: 0.5309
Epoch 4/5
100/100 [==============================] - 5313s 53s/step - loss: 0.6956 - accuracy: 0.5131 - val_loss: 0.7055 - val_accuracy: 0.5309
Epoch 5/5
100/100 [==============================] - 5330s 53s/step - loss: 0.6911 - accuracy: 0.5412 - val_loss: 0.6906 - val_accuracy: 0.5309


In [8]:
model.save_pretrained('../saved_models/model10/')

In [ ]:
filename = '../saved_models/model2.sav'

In [ ]:
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model = pickle.load(open('../saved_models/model3', 'rb'))

In [6]:
model = TFBertForSequenceClassification.from_pretrained('../saved_models/model9/')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ../saved_models/model9/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
for i in range(len(X_test[:100])):
    print(label[i], y_test[i])

In [7]:
tf_batch = tokenizer(X_test[:100], padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
correct, wrong = 0, 0
for i in range(len(X_test[:100])):
  print("predicted:", labels[label[i]], "actual:", labels[int(y_test[i])])
  if labels[label[i]] == labels[int(y_test[i])]:
    correct += 1
  else:
    wrong += 1
print(correct / (correct + wrong))

predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Negative
predicted: Positive actual: Negative
predicted: Positive actual: Positive
predicted: Positive actual: Positive
predicted: Positive actual: Negative
predicted: Positive actual: Negative
p

references from : https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671